# ⚠ Warning

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/OpinionatedGeek%2Fmango-explorer/HEAD?filepath=Retrier.ipynb) _🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

# 🥭 Retrier

This notebook creates a 'retrier' context that can automatically retry failing functions.

In [1]:
import logging
import typing
import requests.exceptions
import time

from contextlib import contextmanager
from decimal import Decimal


# RetryWithPauses class

This class takes a function and a list of durations to pause after a failed call.

If the function succeeds, the resultant value is returned.

If the function fails by raising an exception, the call pauses for the duration at the head of the list, then the head of the list is moved and the function is retried.

It is retried up to the number of entries in the list of delays. If they all fail, the last failing exception is re-raised.

This can be particularly helpful in cases where rate limits prevent further processing.

This class is best used in a `with...` block using the `retry_context()` function below.

In [2]:
class RetryWithPauses:
    def __init__(self, name: str, func: typing.Callable, pauses: typing.List[Decimal]) -> None:
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.name: str = name
        self.func: typing.Callable = func
        self.pauses: typing.List[Decimal] = pauses

    def run(self, *args):
        captured_exception: Exception = None
        for sleep_time_on_error in self.pauses:
            try:
                return self.func(*args)
            except requests.exceptions.HTTPError as exception:
                captured_exception = exception
                if exception.response is not None:
                    # "You will see HTTP respose codes 429 for too many requests
                    # or 413 for too much bandwidth."
                    if exception.response.status_code == 413:
                        self.logger.info(f"Retriable call [{self.name}] rate limited (too much bandwidth) with error '{exception}'.")
                    elif exception.response.status_code == 429:
                        self.logger.info(f"Retriable call [{self.name}] rate limited (too many requests) with error '{exception}'.")
                    else:
                        self.logger.info(f"Retriable call [{self.name}] failed with unexpected HTTP error '{exception}'.")
                else:
                    self.logger.info(f"Retriable call [{self.name}] failed with unknown HTTP error '{exception}'.")
            except Exception as exception:
                self.logger.info(f"Retriable call failed [{self.name}] with error '{exception}'.")
                captured_exception = exception

            if sleep_time_on_error < 0:
                self.logger.info(f"No more retries for [{self.name}] - propagating exception.")
                raise captured_exception

            self.logger.info(f"Will retry [{self.name}] call in {sleep_time_on_error} second(s).")
            time.sleep(float(sleep_time_on_error))

        self.logger.info(f"End of retry loop for [{self.name}] - propagating exception.")
        raise captured_exception


# retry_context generator

This is a bit of Python 'magic' to allow using the Retrier in a `with...` block.

For example, this will call function `some_function(param1, param2)` up to `retry_count` times (7 in this case). It will only retry if the function throws an exception - the result of the first successful call is used to set the `result` variable:
```
pauses = [Decimal(1), Decimal(2), Decimal(4)]
with retry_context("Account Access", some_function, pauses) as retrier:
    result = retrier.run(param1, param2)
```

In [3]:
@contextmanager
def retry_context(name: str, func: typing.Callable, pauses: typing.List[Decimal]) -> typing.Iterator[RetryWithPauses]:
    yield RetryWithPauses(name, func, pauses)


# 🏃 Running

Run a failing method, retrying it 5 times, just to show how it works in practice.

In [4]:
if __name__ == "__main__":
    logging.getLogger().setLevel(logging.INFO)

    def _raiser(value):
        # All this does is raise an exception
        raise Exception(f"This is a test: {value}")

    # NOTE! This will fail by design, with the exception message:
    # "Exception: This is a test: ignored parameter"
#     with retry_context(_raiser, 5) as retrier:
#         response = retrier.run("ignored parameter")

    import rx
    import rx.operators as ops

    from decimal import Decimal
    from rx.scheduler import ThreadPoolScheduler

    from BaseModel import AccountInfo
    from Context import default_context
    from Observables import PrintingObserverSubscriber
    from solana.publickey import PublicKey

    def fetch_group_account(context, pauses):
        def _fetch(_):
            def _actual_fetcher():
                return AccountInfo.load(context, PublicKey("7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV"))
            with retry_context("Group Account Loader", _actual_fetcher, pauses) as retrier:
                return retrier.run()
        return _fetch

    default_context = default_context.new_from_cluster_url("https://api.rpcpool.com")
    pool_scheduler = ThreadPoolScheduler(10)
    rx_subscription = rx.interval(0.01).pipe(
        ops.subscribe_on(pool_scheduler),
        ops.map(fetch_group_account(default_context, [Decimal(1), Decimal(2), Decimal(4), Decimal(8), Decimal(16), Decimal(32)])),
        ops.take(100)
    ).subscribe(PrintingObserverSubscriber(True))


« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9h

2021-05-28 14:32:10 ⓘ RetryWithDel Retriable call [Group Account Loader] rate limited (too many requests) with error '429 Client Error: Too Many Requests for url: https://api.rpcpool.com/'.
2021-05-28 14:32:10 ⓘ RetryWithDel Will retry [Group Account Loader] call in 1 second(s).
2021-05-28 14:32:11 ⓘ RetryWithDel Retriable call [Group Account Loader] rate limited (too many requests) with error '429 Client Error: Too Many Requests for url: https://api.rpcpool.com/'.
2021-05-28 14:32:11 ⓘ RetryWithDel Will retry [Group Account Loader] call in 2 second(s).
2021-05-28 14:32:13 ⓘ RetryWithDel Retriable call [Group Account Loader] rate limited (too many requests) with error '429 Client Error: Too Many Requests for url: https://api.rpcpool.com/'.
2021-05-28 14:32:13 ⓘ RetryWithDel Will retry [Group Account Loader] call in 4 second(s).


« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9h

2021-05-28 14:32:19 ⓘ RetryWithDel Retriable call [Group Account Loader] rate limited (too many requests) with error '429 Client Error: Too Many Requests for url: https://api.rpcpool.com/'.
2021-05-28 14:32:19 ⓘ RetryWithDel Will retry [Group Account Loader] call in 1 second(s).


« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9h

...

« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9h

2021-05-28 14:32:28 ⓘ RetryWithDel Retriable call [Group Account Loader] rate limited (too many requests) with error '429 Client Error: Too Many Requests for url: https://api.rpcpool.com/'.
2021-05-28 14:32:28 ⓘ RetryWithDel Will retry [Group Account Loader] call in 1 second(s).


« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9h

2021-05-28 14:32:30 ⓘ RetryWithDel Retriable call [Group Account Loader] rate limited (too many requests) with error '429 Client Error: Too Many Requests for url: https://api.rpcpool.com/'.
2021-05-28 14:32:30 ⓘ RetryWithDel Will retry [Group Account Loader] call in 1 second(s).


« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
« AccountInfo [7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV]:
    Owner: JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu
    Executable: False
    Lamports: 6069120
    Rent Epoch: 186
»
